# Counting process

In this script, we do some simple calculations of the counting process, seeing the effect of non-exponential waiting times.

In [9]:
%pylab
import scipy.special as ssp
import scipy.signal as scs

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [10]:
def counting_process(rate=10, dist='exp', t_len = int(1e5)):
    tb = np.arange(t_len)/t_len
    
    # approximate number of arrivals
    n_arriv = int(10*rate)
    
    # waiting times
    if dist=='exp':
        tau = np.random.exponential(1/rate, size=n_arriv)
    elif dist=='ray':
        mean = 1/rate
        par = np.sqrt(2/np.pi)*mean
        tau = np.random.rayleigh(par, size=n_arriv)
    elif dist == 'deg':
        tau = np.abs(np.random.normal(loc=1, scale=1e-3, size=n_arriv))/rate 
        #tau = np.ones(n_arriv)/rate
    elif dist == 'gamma10':
        k = 10.
        mean = 1./rate
        A = mean/k
        tau = np.random.gamma(k, scale = A, size=n_arriv)
    elif dist =='pareto':
        a = 1.5
        param = 1./(3.*rate)
        tau = (np.random.pareto(a, size=n_arriv) +1)*param
    # arrival times
    arrivals = np.cumsum(tau)
    arrivals = arrivals[arrivals<=1]
    
    # the counting process
    K = np.zeros(tb.size)
    for i in range(arrivals.size):
        K[int(arrivals[i]*t_len):] += 1
    
    return tb, K, arrivals

In [11]:
def plot_tau_dist():
    rate = 1.
    n_arriv = int(1e6)
    
    for dist in ['exp', 'ray', 'deg', 'pareto']:
        # waiting times
        if dist=='exp':
            tau = np.random.exponential(1/rate, size=n_arriv)
            symb = 'o'
        elif dist=='ray':
            mean = 1/rate
            par = np.sqrt(2/np.pi)*mean
            tau = np.random.rayleigh(par, size=n_arriv)
            symb = 'd'
        elif dist == 'deg':
            tau = np.abs(np.random.normal(loc=1, scale=1e-1, size=n_arriv))/rate 
            symb = 'x'
            #tau = np.ones(n_arriv)/rate
        elif dist == 'gamma10':
            k = 10.
            mean = 1./rate
            A = mean/k
            tau = np.random.gamma(k, scale = A, size=n_arriv)
            symb = '*'
        
        elif dist =='pareto':
            a = 1.5
            param = 1./(3.*rate)
            tau = (np.random.pareto(a, size=n_arriv) +1)*param
            symb = '+'
        print(tau.mean())
        hist, be = np.histogram(tau, 64, density=True)
        bins = (be[1:]+be[:-1])/2
        
        plt.figure('tau dist')
        plt.plot(bins, hist, symb, label=dist)
    plt.xlabel('tau')
    plt.ylabel('p_tau')
    plt.legend()
    
plot_tau_dist()

0.9994326025384219
0.9995797186595056
1.000031537256575
0.9939989377375303


In [12]:
def plot_counts(rate=10, dist='exp'):
    tb, K, arrivals = counting_process(rate = rate, dist=dist)

    
    plt.figure('counts')
    plt.plot(tb, K, 'C0', label='K(t)')
    if rate<=100:
        plt.plot(arrivals, np.arange(arrivals.size)+1, 'C1o', label='t_i')
    plt.plot(tb, tb*rate, 'C2',label='<K(t)>')
    plt.xlabel('t')
    plt.ylabel('# events')
    plt.legend()

plot_counts(rate=10, dist='deg')

In [13]:
def plot_Kdist(rate=10, dist = 'exp', repeat=1000):
    Karr = np.zeros(repeat)
    
    for i in range(repeat):
        tb, K, arrivals = counting_process(rate = rate, dist=dist)
        Karr[i] = K[-1]

    Kmax = 25
    Kbin_edges = np.arange(Kmax+2)-0.5
    Kbin = np.arange(Kmax+1)
    Khist, _ = np.histogram(Karr, bins=Kbin_edges, density=True)
    
    plt.figure('K pdf')
    Kpdf = np.zeros(Kbin.size)
    for k in range(Kbin.size):
        Kpdf[k] = rate**Kbin[k]*np.exp(-rate)/ssp.factorial(Kbin[k])
    plt.plot(Kbin, Kpdf, '-o', label='Poisson')
    plt.plot(Kbin, Khist, 'o', label='empirical')
    plt.xlabel('k')
    plt.ylabel('p_K(k)')
    plt.legend()
    
plot_Kdist(rate=10., dist='pareto', repeat=1000)

In [15]:
def plot_arrival_stat(rate=10, dist='exp'):
    plt.close('all')
    tb, K, arrivals = counting_process(rate = rate, dist=dist, t_len=int(1e6))
    
    plt.figure('test')
    plt.plot(tb, K)
    plt.plot(tb, tb*rate)
    plt.plot(tb[1:], K[1:]-K[:-1])

    hist, edge = np.histogram(arrivals, bins=10, density=True)
    
    bins = (edge[1:]+edge[:-1])/2
    
    plt.figure('arrival pmf')
    plt.plot(bins, np.ones(bins.size), label='true')
    plt.plot(bins, hist, 'o', label='approx')
    plt.xlabel('t')
    plt.ylabel('p_t_i')
    plt.legend()
    plt.axis([0,1,0,1.5])
    
    plt.figure('arrival corrf')
    aproc = K[1:]-K[:-1]
    na = (aproc-np.mean(aproc))/np.std(aproc)
    R = scs.correlate(na, na, mode='full')
    
    t = np.arange(-(na.size-1), na.size)
    R /= (na.size-np.abs(t))
    plt.plot(t,R)#, '-o')
    plt.xlabel('n')
    plt.ylabel('R(t_i, t_(n+i))')
    #plt.axis([-200, 200, -0.1,1.1])
    
    naF = np.fft.rfft(na)
    naS = np.real(naF*np.conj(naF))
    f = np.fft.rfftfreq(na.size, 1e-5)
    plt.figure('arrival PSD lin')
    plt.plot(f, naS)

    plt.figure('arrival PSD loglog')
    plt.loglog(f, naS)
    
plot_arrival_stat(rate=10000, dist='deg')